In [1]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from tempfile import mkdtemp
import warnings

c:\Users\ivant\anaconda3\envs\capstone\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ivant\anaconda3\envs\capstone\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
c:\Users\ivant\anaconda3\envs\capstone\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
numerical_df = pd.read_csv("numerical_df.csv")

In [3]:
pd.set_option('display.max_columns', None) 
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average,PER Improvement
0,A.C. Green,23.0,79.0,72.0,28.4,0.538,0.1,0.000,7.4,0.543,0.538,3.6,0.780,2.7,7.8,1.1,0.9,1.0,1.3,2.2,10.8,LAL,1987/1988,PF,6-9,220.0,1.0,15.7,0.599,0.009,4.6,1.5,2.0,12.5,14.7,4.3,7.6,0.163,1.0,0.9,1.7,15.3,205.74,F,0.0,1.0,0.0,13.986667,1.2,0.300000,0.5,0.006667,0.206667,0.539867,9.453333,0.0,14.540000,0.054467,0.481200,-1.2
1,A.C. Green,24.0,82.0,64.0,32.1,0.503,0.0,0.000,7.8,0.505,0.503,4.6,0.773,3.0,8.7,1.1,1.1,0.5,1.5,2.5,11.4,LAL,1988/1989,PF,6-9,220.0,2.0,14.5,0.581,0.003,4.5,1.6,1.0,12.9,14.7,4.5,7.9,0.144,0.3,0.4,1.5,15.3,205.74,F,0.0,1.0,0.0,13.950000,-3.3,0.235714,-1.4,-0.021429,0.185714,0.536929,9.314286,-2.3,14.528571,0.058143,0.479643,3.3
2,A.C. Green,25.0,82.0,82.0,30.6,0.529,0.2,0.235,9.0,0.536,0.532,4.4,0.786,3.1,9.0,1.3,1.1,0.7,1.5,2.1,13.3,LAL,1989/1990,PF,6-9,220.0,3.0,17.8,0.594,0.022,5.5,1.8,1.2,11.5,17.0,5.8,9.4,0.179,1.6,1.8,2.3,16.4,205.74,F,0.0,1.0,0.0,13.653846,3.1,0.507692,1.5,-0.161538,0.307692,0.532538,9.007692,-0.1,14.338462,0.060923,0.475846,-3.1
3,A.C. Green,26.0,82.0,82.0,33.0,0.478,0.6,0.283,9.3,0.489,0.486,4.5,0.751,3.2,8.7,1.1,0.8,0.6,1.4,2.5,12.9,LAL,1990/1991,PF,6-9,220.0,4.0,14.7,0.548,0.057,4.6,1.2,1.1,10.7,17.1,4.4,7.7,0.137,-0.6,0.3,1.0,15.1,205.74,F,0.0,1.0,0.0,13.566667,0.9,0.291667,0.2,-0.200000,0.208333,0.531250,8.683333,1.5,14.108333,0.061250,0.475667,-0.9
4,A.C. Green,27.0,82.0,21.0,26.4,0.476,0.7,0.200,5.9,0.507,0.486,3.7,0.738,2.5,6.3,0.9,0.7,0.3,1.2,1.4,9.1,LAL,1991/1992,PF,6-9,220.0,5.0,13.8,0.556,0.101,4.5,1.4,0.6,12.8,15.6,3.2,6.1,0.135,0.0,0.1,1.1,13.8,205.74,F,0.0,1.0,0.0,13.545455,-2.9,0.236364,-1.6,-0.227273,0.209091,0.529000,8.645455,-1.2,13.972727,0.057636,0.475636,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11389,Žarko Čabarkapa,23.0,37.0,0.0,12.5,0.482,1.0,0.361,3.6,0.515,0.521,1.7,0.813,0.9,2.7,0.7,0.3,0.1,0.8,1.6,6.2,GSW,2005/2006,PF,6-11,235.0,1.0,16.5,0.578,0.212,9.2,1.1,0.8,13.1,21.7,0.8,1.2,0.124,-0.5,0.1,0.2,11.6,210.82,F,0.0,1.0,0.0,16.500000,0.0,0.000000,0.0,0.100000,0.000000,0.578000,6.200000,0.0,21.700000,0.212000,0.482000,-6.6
11390,Željko Rebrača,29.0,74.0,4.0,15.9,0.505,0.0,0.000,5.1,0.505,0.505,2.4,0.771,1.1,3.9,0.5,0.4,1.0,1.1,2.6,6.9,DET,2002/2003,C,7-0,257.0,0.0,16.2,0.569,0.000,5.7,1.3,4.4,15.7,21.4,1.4,3.3,0.134,-2.3,-2.9,-0.1,14.5,213.36,C,1.0,0.0,0.0,12.350000,5.5,1.100000,0.8,-2.700000,-0.600000,0.608500,5.250000,5.9,15.750000,0.000000,0.555000,-2.2
11391,Željko Rebrača,30.0,30.0,12.0,16.3,0.552,0.0,0.000,4.8,0.552,0.552,1.6,0.792,0.9,3.1,0.3,0.2,0.6,1.0,2.6,6.6,DET,2003/2004,C,7-0,257.0,1.0,14.0,0.596,0.000,3.6,0.7,2.6,14.9,19.2,0.7,1.3,0.133,-2.6,-2.8,-0.1,11.2,213.36,C,1.0,0.0,0.0,13.633333,-2.2,0.000000,0.1,-2.766667,-0.366667,0.595333,5.800000,-2.2,17.633333,0.000000,0.538333,-4.4
11392,Željko Rebrača,31.0,21.0,2.0,10.6,0.407,0.0,0.000,2.6,0.407,0.407,1.3,0.786,0.9,2.3,0.2,0.2,0.4,0.6,2.1,3.1,DET,2004/2005,C,7-0,257.0,2.0,9.6,0.498,0.000,3.0,1.2,3.1,16.4,16.7,0.1,0.5,0.115,-3.9,-5.1,-0.1,12.8,213.36,C,1.0,0.0,0.0,13.725000,-4.4,-1.100000,-2.3,-2.775000,-0.850000,0.595500,6.000000,-2.5,18.025000,0.000000,0.541750,4.4


In [4]:
# Save metadata for later use
metadata_columns = ['Player', 'season', 'team', 'Age', 'PER']
metadata = numerical_df[metadata_columns]

# Identify object columns to be dropped after splitting
object_cols_to_drop_after_split = ['Player', 'season', 'team']

# Separate features and the target variable
features = numerical_df.drop(['PER Improvement'], axis=1)
target = numerical_df['PER Improvement']

# Isolate the test set for the season 2023/2024 using 'season' but don't drop it yet
test_indices = numerical_df['season'] == '2023/2024'
train_indices = ~test_indices

# Create training and testing datasets
X_train = features[train_indices]
y_train = target[train_indices]
X_test = features[test_indices]
y_test = target[test_indices]

# Save test metadata for results later
metadata_test = metadata[test_indices].reset_index(drop=True)

# Drop object columns that are not needed for modeling
X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])

In [5]:
y_test

16       -1.0
21      -11.4
42       -2.7
49        2.1
64        2.3
         ... 
11175   -12.1
11204     1.2
11327    -2.1
11336    -3.9
11362    -0.1
Name: PER Improvement, Length: 310, dtype: float64

In [6]:
warnings.filterwarnings('ignore')
cachedir = mkdtemp()

estimators = [('normalise', StandardScaler()),   
            #('reduce_dim', PCA(n_components = 40)),
              ('xgb', XGBRegressor(random_state=1))]
pipe = Pipeline(estimators, memory = cachedir) 

param_grid = {
    'xgb__n_estimators': [590],
    'xgb__max_depth': [7],
    'xgb__learning_rate': [0.01],
    'xgb__min_child_weight': [0.001],
    'xgb__subsample': [0.4]
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=5, refit=True)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END xgb__learning_rate=0.01, xgb__max_depth=7, xgb__min_child_weight=0.001, xgb__n_estimators=590, xgb__subsample=0.4; total time=   2.4s
[CV 2/5] END xgb__learning_rate=0.01, xgb__max_depth=7, xgb__min_child_weight=0.001, xgb__n_estimators=590, xgb__subsample=0.4; total time=   2.1s
[CV 3/5] END xgb__learning_rate=0.01, xgb__max_depth=7, xgb__min_child_weight=0.001, xgb__n_estimators=590, xgb__subsample=0.4; total time=   2.2s
[CV 4/5] END xgb__learning_rate=0.01, xgb__max_depth=7, xgb__min_child_weight=0.001, xgb__n_estimators=590, xgb__subsample=0.4; total time=   2.7s
[CV 5/5] END xgb__learning_rate=0.01, xgb__max_depth=7, xgb__min_child_weight=0.001, xgb__n_estimators=590, xgb__subsample=0.4; total time=   2.5s


GridSearchCV(cv=5,
             estimator=Pipeline(memory='C:\\Users\\ivant\\AppData\\Local\\Temp\\tmph8xnryvk',
                                steps=[('normalise', StandardScaler()),
                                       ('xgb',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     device=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_metric=None,
                                    

In [7]:
from sklearn.metrics import accuracy_score
best_params = grid_search.best_params_  # Getting the best parameters from our grid search
print(f"Best parameters: {best_params}") 
XG_best_pipe = grid_search.best_estimator_

Best parameters: {'xgb__learning_rate': 0.01, 'xgb__max_depth': 7, 'xgb__min_child_weight': 0.001, 'xgb__n_estimators': 590, 'xgb__subsample': 0.4}


In [8]:
y_test_pred = XG_best_pipe.predict(X_test)  # make predictions on training set 

best_validation_performance = grid_search.best_score_  # Getting the best validation set score from our grid search
print(f"Best validation performance: {best_validation_performance}")

y_test_pred = XG_best_pipe.predict(X_test)  # make predictions on training set 

r2_test = r2_score(y_test, y_test_pred)  # get r2 
print(f'R squared: {round(r2_test, 3)}')

rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))  # get rmse 
print(f'RMSE: {round(rmse_test, 2)}')

mae_test = mean_absolute_error(y_test, y_test_pred)  # get mae
print(f'MAE: {round(mae_test, 2)}')


Best validation performance: 0.3615746766622213
R squared: 0.109
RMSE: 2.96
MAE: 2.1


In [9]:
predictions_df = pd.DataFrame({
    'Predicted PER Improvement': y_test_pred
    'Actual PER Improvement': y_test,
}).reset_index(drop=True)

# Combine the predictions with the metadata
final_df = pd.concat([metadata_test, predictions_df], axis=1)

# Calculate the error
final_df['Error'] = final_df['Predicted PER Improvement'] - final_df['Actual PER Improvement']

# Display the DataFrame
final_df

,Player,season,team,Age,PER,Actual PER Improvement,Predicted PER Improvement,Error
0,A.J. Green,2023/2024,MIL,23.0,11.5,-1.0,-0.987780,0.012220
1,AJ Griffin,2023/2024,ATL,19.0,12.6,-11.4,0.153691,11.553691
2,Aaron Gordon,2023/2024,DEN,27.0,19.5,-2.7,-2.511815,0.188185
3,Aaron Holiday,2023/2024,ATL,26.0,9.4,2.1,0.813880,-1.286120
4,Aaron Nesmith,2023/2024,IND,23.0,10.5,2.3,0.403935,-1.896065
...,...,...,...,...,...,...,...,...
305,Wenyen Gabriel,2023/2024,LAL,25.0,14.4,-12.1,-0.296737,11.803263
306,Wesley Matthews,2023/2024,MIL,36.0,7.0,1.2,0.864108,-0.335892
307,Zach Collins,2023/2024,SAS,25.0,16.4,-2.1,-0.694634,1.405366
308,Zach LaVine,2023/2024,CHI,27.0,19.0,-3.9,-0.170035,3.729965


In [10]:
overestimation_percentage = (final_df['Error'] > 0).mean() * 100
underestimation_percentage = (final_df['Error'] < 0).mean() * 100

print(f"Overestimation Percentage: {overestimation_percentage}%")
print(f"Underestimation Percentage: {underestimation_percentage}%")

Overestimation Percentage: 49.67741935483871%
Underestimation Percentage: 50.32258064516129%


In [11]:
final_df['Correct Prediction of Improvement'] = (
    (final_df['Predicted PER Improvement'] > 0) & (final_df['Actual PER Improvement'] > 0) |
    (final_df['Predicted PER Improvement'] < 0) & (final_df['Actual PER Improvement'] < 0)
)

# Calculate total actual improvements and correct predictions
total_actual_changes = ((final_df['Actual PER Improvement'] > 0) | (final_df['Actual PER Improvement'] < 0)).sum()
correct_predictions_of_changes = final_df['Correct Prediction of Improvement'].sum()

# Calculate the percentage of correct predictions
if total_actual_changes > 0:
    correct_prediction_percentage = (correct_predictions_of_changes / total_actual_changes) * 100
else:
    correct_prediction_percentage = 0  # To handle cases where there are no actual changes

print(f"Percentage of correctly predicting a player's performance change (improvement or decline) when the player actually changed: {correct_prediction_percentage}%")

Percentage of correctly predicting a player's performance change (improvement or decline) when the player actually changed: 67.54098360655738%


In [23]:
final_df.rename(columns={'PER': '2022/2023 PER'}, inplace=True)
final_df.rename(columns={'Error': 'Prediction Difference'}, inplace=True)


In [29]:
final_df["2023/2024 PER"] = final_df["2022/2023 PER"] + final_df["Actual PER Improvement"]
column_order = [
    'Player', 'season', 'team', 'Age', '2022/2023 PER', '2023/2024 PER', 
    'Actual PER Improvement', 'Predicted PER Improvement', 'Correct Prediction of Improvement', 'Prediction Difference']

final_df = final_df[column_order]
final_df["Age"] +=1

In [30]:
final_df

,Player,season,team,Age,2022/2023 PER,2023/2024 PER,Actual PER Improvement,Predicted PER Improvement,Correct Prediction of Improvement,Prediction Difference
0,A.J. Green,2023/2024,MIL,24.0,11.5,10.5,-1.0,-0.987780,True,0.012220
1,AJ Griffin,2023/2024,ATL,20.0,12.6,1.2,-11.4,0.153691,False,11.553691
2,Aaron Gordon,2023/2024,DEN,28.0,19.5,16.8,-2.7,-2.511815,True,0.188185
3,Aaron Holiday,2023/2024,ATL,27.0,9.4,11.5,2.1,0.813880,True,-1.286120
4,Aaron Nesmith,2023/2024,IND,24.0,10.5,12.8,2.3,0.403935,True,-1.896065
...,...,...,...,...,...,...,...,...,...,...
305,Wenyen Gabriel,2023/2024,LAL,26.0,14.4,2.3,-12.1,-0.296737,True,11.803263
306,Wesley Matthews,2023/2024,MIL,37.0,7.0,8.2,1.2,0.864108,True,-0.335892
307,Zach Collins,2023/2024,SAS,26.0,16.4,14.3,-2.1,-0.694634,True,1.405366
308,Zach LaVine,2023/2024,CHI,28.0,19.0,15.1,-3.9,-0.170035,True,3.729965


In [32]:
final_df.loc[final_df["Player"] == "Stephen Curry"]

,Player,season,team,Age,2022/2023 PER,2023/2024 PER,Actual PER Improvement,Predicted PER Improvement,Correct Prediction of Improvement,Prediction Difference
276,Stephen Curry,2023/2024,GSW,35.0,24.1,20.6,-3.5,-3.487025,True,0.012975


In [33]:
import pickle

filename = "xgboost.pkl"

with open(filename, "wb") as file:
    pickle.dump(XG_best_pipe, file)

In [34]:
with open(filename, "rb") as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test)
predictions

array([-9.8778003e-01,  1.5369061e-01, -2.5118151e+00,  8.1388021e-01,
        4.0393478e-01, -1.2385279e+00, -2.3132873e+00, -2.8056216e+00,
       -4.7267711e-01,  8.3692968e-01,  4.8277850e+00, -1.9062167e+00,
        5.5980247e-01, -1.0426058e+00, -5.2724326e-01, -2.3119814e+00,
        1.1262243e+00, -1.6840644e+00,  9.5179307e-01,  5.4862994e-01,
        1.5322155e+00, -3.5015798e-01,  3.0947299e+00, -3.5463939e+00,
       -1.8793577e-01, -1.2335968e+00, -1.8804240e+00,  3.1735677e-01,
       -9.0162909e-01, -2.3311653e+00, -2.7507122e+00, -1.4926229e-01,
       -2.0371654e+00, -1.9946667e+00,  1.5982631e+00,  1.0244610e+00,
        5.1949745e-01,  8.8964003e-01, -2.7434555e-01, -6.2523586e-01,
       -7.1675837e-01, -1.5320446e+00, -2.6296115e+00,  3.1036336e+00,
       -1.0827510e+00,  7.2860622e-01, -2.1068532e+00,  2.5325065e+00,
       -2.2831764e+00,  4.8899144e-02, -6.3848752e-01, -7.0374680e-01,
       -1.6007949e+00, -3.1821650e-02,  3.2227473e+00, -3.0376279e-01,
      

In [35]:
final_df.to_csv("final_df.csv", index = False)